In [1]:
import pandas as pd

Source: http://dashboard.healthit.gov/datadashboard/documentation/ehr-products-mu-attestation-data-documentation.php . These are the 2015 data only. See this page for variable names and explanations.

Notably, each row in this csv corresponds to not a single medical center, but an attestation to the use of a particular EHR product. Therefore, as part of each medical center's meaningful use attestation to the CMS (which has a single unique `Attestation_ID`), they may attest to the use of multiple certified products within that single attestation.

In [51]:
full_df = pd.read_csv('MU_REPORT_2015.csv', encoding='Latin1') # note the encoding. it wouldn't read with utf-8

### Tasks to convert to sql/clean data

- [*Done*] lowercase+snake_case all column labels (to make adding to db neater)
- [*Done*] Fix `NaN`s in `dtype('O')` columns
- Re-export as cleaned csv, or [*Done*] import into db directly from pd

In [52]:
full_df.columns = [col.lower().replace('.', '_') for col in full_df.columns]

In [53]:
full_df[full_df['specialty'].isnull()].shape[0]

18581

Column 'Specialty' includes `NaN`s as well as "UNKNOWN". It's probably fair game to cast empty cells (read as `NaN`) as unknown, so we can treat this as a column of exclusively object dtype, not mixed.

In [56]:
full_df.loc[full_df['specialty'].isnull(), 'specialty'] = 'UNKNOWN'

In [57]:
full_df[full_df['specialty'].isnull()].shape[0]

0

In [75]:
from sqlalchemy import create_engine
from getpass import getuser

engine = create_engine('postgres://{}@localhost/ehr_mu'.format(getuser()))  # note: need pip-installed psycopg2

In [76]:
full_df.to_sql('attest_2015', engine)

In [54]:
full_df.dtypes

ehr_certification_number             object
vendor_name                          object
ehr_product_chp_id                   object
ehr_product_name                     object
ehr_product_version                  object
product_classification               object
product_setting                      object
product_certification_edition_yr      int64
attestation_month                     int64
attestation_year                      int64
business_state_territory             object
provider_type                        object
specialty                            object
program_year                          int64
payment_year                        float64
provider_stage_number                object
program_type                         object
stage_2_scheduled_2014              float64
mu_definition_2014                  float64
attestation_id                        int64
npi                                   int64
ccn                                 float64
dtype: object

In [60]:
for col in full_df.columns:
    to_print = str(full_df[col].dtype) if full_df[col].dtype != 'O' else str(max(full_df[col].apply(len)))
    print(col + ': ' + to_print)

ehr_certification_number: 15
vendor_name: 59
ehr_product_chp_id: 10
ehr_product_name: 223
ehr_product_version: 45
product_classification: 12
product_setting: 10
product_certification_edition_yr: int64
attestation_month: int64
attestation_year: int64
business_state_territory: 30
provider_type: 8
specialty: 36
program_year: int64
payment_year: float64
provider_stage_number: 7
program_type: 17
stage_2_scheduled_2014: float64
mu_definition_2014: float64
attestation_id: int64
npi: int64
ccn: float64


In [67]:
full_df['attestation_month'].value_counts()

2    171358
1    107324
5     71382
3     69840
4      1422
6       990
7       117
8        96
Name: attestation_month, dtype: int64

In [72]:
test = full_df['attestation_month'].apply(str) + '/' + full_df['attestation_year'].apply(str)
test.value_counts()

2/2016    171358
1/2016    107324
5/2016     71382
3/2016     69840
4/2016      1422
6/2016       990
7/2015       111
8/2015        96
7/2016         6
dtype: int64

In [63]:
full_df['ehr_product_chp_id'].apply(len).nunique()

1

In [73]:
full_df['provider_type'].value_counts()

EP          403948
Hospital     18581
Name: provider_type, dtype: int64

In [3]:
full_df.head()

,EHR.Certification.Number,Vendor.Name,EHR.Product.CHP.Id,EHR.Product.Name,EHR.Product.Version,Product.Classification,Product.Setting,Product.Certification.Edition.Yr,Attestation.Month,Attestation.Year,...,Specialty,Program.Year,Payment.Year,Provider.Stage.Number,Program.Type,Stage.2.Scheduled.2014,MU.Definition.2014,Attestation_ID,NPI,CCN
0,A014E01NCEULEA5,Epic Systems Corporation,CHP-022044,EpicCare Ambulatory 2014 Certified EHR Suite,Epic 2014 (June 2013 CQMs),Complete EHR,Ambulatory,2014,2,2016,...,UNKNOWN,2015,2.0,Stage 1,Medicare,0.0,NaN,1,1003000142,NaN
1,1314E01PHNJKEA5,NextGen Healthcare,CHP-023369,NextGen Ambulatory EHR,5.8.1,Complete EHR,Ambulatory,2014,2,2016,...,UNKNOWN,2015,4.0,Stage 2,Medicare,1.0,NaN,4,1003000522,NaN
2,1314E01PDRTLEAR,Get Real Health,CHP-019270,InstantPHR,3,Modular EHR,Ambulatory,2014,2,2016,...,UNKNOWN,2015,4.0,Stage 2,Medicare,1.0,NaN,6,1003000530,NaN
3,1314E01PDRTLEAR,Allscripts,CHP-022204,Allscripts TouchWorks EHR,11.4.1,Modular EHR,Ambulatory,2014,2,2016,...,UNKNOWN,2015,4.0,Stage 2,Medicare,1.0,NaN,6,1003000530,NaN
4,1314E01PHNJKEA5,NextGen Healthcare,CHP-023369,NextGen Ambulatory EHR,5.8.1,Complete EHR,Ambulatory,2014,2,2016,...,UNKNOWN,2015,NaN,Stage 1,Medicare,0.0,NaN,10,1003000597,NaN


In [3]:
full_df.columns

Index(['EHR.Certification.Number', 'Vendor.Name', 'EHR.Product.CHP.Id',
       'EHR.Product.Name', 'EHR.Product.Version', 'Product.Classification',
       'Product.Setting', 'Product.Certification.Edition.Yr',
       'Attestation.Month', 'Attestation.Year', 'Business.State.Territory',
       'Provider.Type', 'Specialty', 'Program.Year', 'Payment.Year',
       'Provider.Stage.Number', 'Program.Type', 'Stage.2.Scheduled.2014',
       'MU.Definition.2014', 'Attestation_ID', 'NPI', 'CCN'],
      dtype='object')

In [4]:
vendor_attestation_counts = full_df['Vendor.Name'].value_counts()
single_attestation_vendors = vendor_attestation_counts[vendor_attestation_counts==1]
print('total unique vendors: ' + str(len(vendor_attestation_counts)))
print('count vendors with single attestation: ' + str(len(single_attestation_vendors)))
single_attestation_vendors
# vendor_attestation_counts

total unique vendors: 477
count vendors with single attestation: 47


ALERT Life Sciences Computing SA                              1
Amrita Ventures LLC                                           1
LCD Solutions Inc/Clicktate                                   1
OSEHRA                                                        1
Meta Pharmacy Systems Inc dba Meta Healthcare IT Solutions    1
Brigham and Womens Hospital                                   1
Nth Technologies Inc                                          1
Panacea Medical MBA                                           1
Medicat LLC                                                   1
Sajix Inc                                                     1
Hill-Rom Holdings Inc                                         1
Paragon Billing LLC                                           1
Metasolutions Inc                                             1
Twin Sails Technology Group Inc                               1
Draeger Medical Systems Inc                                   1
Nuance Communications Inc               

In [61]:
full_df['product_classification'].value_counts(dropna=False)

Complete EHR    225515
Modular EHR     197014
Name: product_classification, dtype: int64

Interesting: not all the single attestation vendors are hospitals doing their own thing. Some are simply small providers?

In [97]:
vendor_attestation_counts.sort_values(ascending=False).head()

Epic Systems Corporation    121058
Cerner Corporation          104928
Allscripts                   28230
eClinicalWorks LLC           18931
NextGen Healthcare           13705
Name: Vendor.Name, dtype: int64

In [5]:
print(full_df[full_df['Vendor.Name']=="ALERT Life Sciences Computing SA"].NPI)
print(full_df[full_df['Vendor.Name']=="ALERT Life Sciences Computing SA"].CCN)

238386    1568543221
Name: NPI, dtype: int64
238386   NaN
Name: CCN, dtype: float64


Let's confirm that having an NPI and having a CCN is mutually exclusive

In [6]:
print(len(full_df[full_df.NPI.isnull()]))
print(len(full_df[full_df.CCN.notnull()]))

0
18581


Apparently not. Looks like everyone has an NPI, but only facilities that participate in federal health care programs (whatever that means) have a CCN. I'm okay with this, because the database that holds all further information about NPIs, the NPPES (https://npiregistry.cms.hhs.gov/) provides an API! Hooray! This means we're not reliant on the simple state information given in the EHR csv...we can get really specific business addresses for each provider! The JSON returned by the NPPES API even breaks down address for you, so there's no parsing required if the level of granularity I'm interested in is at the city or zip code level.

In [7]:
import requests, json

In [27]:
test_api_req = requests.get('https://npiregistry.cms.hhs.gov/api/', params={'number':1568543221})
test_json = json.loads(test_api_req.text)

In [28]:
test_json['results'][0]['addresses'][0]['postal_code']

'787333233'

We could use the Google Maps API to find the exact lat/long of this address, but the postal code is really more than enough detail, so let's try another API. The zippopotamus API is nicely open-source (https://github.com/ekotechnology/Zippopotamus-Cloud, bonus points for Python) and rather permissively-licensed under the Open Database License (http://opendatacommons.org/licenses/odbl/1.0/). It's got the quirk of not accepting the +4 part of zip codes, but again, this is far beyond our desired level of granularity. 

The larger annoyance is the API requiring constructed urls, rather than accepting query parameters. Could this be somewhere I could contribute?

In [10]:
r = requests.get('https://api.zippopotam.us/us/' + (test_json['results'][0]['addresses'][0]['postal_code'][:5]))
test_json2 = json.loads(r.text)

In [11]:
print(float(test_json2['places'][0]['latitude']))
test_json2

30.3314


{'country': 'United States',
 'country abbreviation': 'US',
 'places': [{'latitude': '30.3314',
   'longitude': '-97.8666',
   'place name': 'Austin',
   'state': 'Texas',
   'state abbreviation': 'TX'}],
 'post code': '78733'}

The zippopotamus documentation (sparse as it is) seems to assert a single place per zip code (in contrast to the multiple places per postal code case for other countries), so we should be OK using `['places'][0]`.

Ok. The next task is to add lat and long columns to the df. Note that we can nearly halve the number of requests by not repeating requests for NPIs we've already looked up.

In [72]:
# NPI_loc_dict = {}
def get_lat_long(row):
    if row.NPI not in NPI_loc_dict:
        NPI_info = requests.get('https://npiregistry.cms.hhs.gov/api/', params={'number':row.NPI}).text
        NPI_zip = json.loads(NPI_info)['results'][0]['addresses'][0]['postal_code'][:5]
        zip_info = json.loads(requests.get('https://api.zippopotam.us/us/' + NPI_zip).text)['places'][0]
    # to test, try printing them
#     print(zip_info['latitude'] + ', ' + zip_info['longitude'])

        NPI_loc_dict[row.NPI] = {'lat': float(zip_info['latitude']), 
                                 'long': float(zip_info['longitude'])}
#     else:
#         print('lookup worked')
    

test_df = full_df.head()
test_df.apply(get_lat_long, axis=1)
# just ignore the warning, it seems to be working well enough.

0    None
1    None
2    None
3    None
4    None
dtype: object

In [66]:
NPI_loc_dict

{1003000142: {'lat': 41.708, 'long': -83.6434},
 1003000522: {'lat': 28.8989, 'long': -81.2473},
 1003000530: {'lat': 40.4411, 'long': -75.3507},
 1003000597: {'lat': 36.1464, 'long': -95.9526}}

In [75]:
test2 = pd.DataFrame.from_dict(NPI_loc_dict, orient='index')
test2

,lat,long
1003000142,41.7080,-83.6434
1003000522,28.8989,-81.2473
1003000530,40.4411,-75.3507
1003000597,36.1464,-95.9526


In [73]:
test_df = pd.merge(test_df,test2, left_on='NPI', right_index=True)  # assume inner join
test_df

,EHR.Certification.Number,Vendor.Name,EHR.Product.CHP.Id,EHR.Product.Name,EHR.Product.Version,Product.Classification,Product.Setting,Product.Certification.Edition.Yr,Attestation.Month,Attestation.Year,...,Payment.Year,Provider.Stage.Number,Program.Type,Stage.2.Scheduled.2014,MU.Definition.2014,Attestation_ID,NPI,CCN,lat,long
0,A014E01NCEULEA5,Epic Systems Corporation,CHP-022044,EpicCare Ambulatory 2014 Certified EHR Suite,Epic 2014 (June 2013 CQMs),Complete EHR,Ambulatory,2014,2,2016,...,2.0,Stage 1,Medicare,0.0,NaN,1,1003000142,NaN,41.7080,-83.6434
1,1314E01PHNJKEA5,NextGen Healthcare,CHP-023369,NextGen Ambulatory EHR,5.8.1,Complete EHR,Ambulatory,2014,2,2016,...,4.0,Stage 2,Medicare,1.0,NaN,4,1003000522,NaN,28.8989,-81.2473
2,1314E01PDRTLEAR,Get Real Health,CHP-019270,InstantPHR,3,Modular EHR,Ambulatory,2014,2,2016,...,4.0,Stage 2,Medicare,1.0,NaN,6,1003000530,NaN,40.4411,-75.3507
3,1314E01PDRTLEAR,Allscripts,CHP-022204,Allscripts TouchWorks EHR,11.4.1,Modular EHR,Ambulatory,2014,2,2016,...,4.0,Stage 2,Medicare,1.0,NaN,6,1003000530,NaN,40.4411,-75.3507
4,1314E01PHNJKEA5,NextGen Healthcare,CHP-023369,NextGen Ambulatory EHR,5.8.1,Complete EHR,Ambulatory,2014,2,2016,...,NaN,Stage 1,Medicare,0.0,NaN,10,1003000597,NaN,36.1464,-95.9526


In [92]:
test_df[test_df['Vendor.Name'].str.contains('epic',case=False)].shape[0]

1

OK, it works! Now we have to apply this to nearly a quarter million rows, and then figure out what to do with it. I'll decide on a question first, since I may only be interested in the locations of the top few vendors.

Actually, let's brainstorm here. I want to see whether the prevalence of a particular vendor varies with either lat or long. Does that mean I need bins? Is there a way to do this on a continuous scale? Maybe let me try to plot prevalence by a predetermined coarse bin -- states.

In [95]:
AllEpic = full_df[full_df['Vendor.Name'].str.contains('epic', case=False)]
AllEpic['Vendor.Name'].value_counts()

Epic Systems Corporation    121058
Name: Vendor.Name, dtype: int64

In [99]:
state_groups = full_df.groupby('Business.State.Territory')
# for each group, compute a proportion of Epic attestations
def get_epic_prop(df):
    nEpic = len(df[df['Vendor.Name'].str.contains('epic',case=False)])
    nTotal = len(df)
    return nEpic/nTotal

for name, group in state_groups:
    print(name + ': ' + str(get_epic_prop(group)))
    

Alabama: 0.023890142964635062
Alaska: 0.09765625
Arizona: 0.0272756105296543
Arkansas: 0.2351138353765324
California: 0.35941979522184303
Colorado: 0.3320173707066719
Connecticut: 0.30270692347735556
Delaware: 0.0006472491909385113
District Of Columbia: 0.006130790190735695
Federated States Of Micronesia: 0.0
Florida: 0.14714577321038924
Georgia: 0.11606280193236715
Guam: 0.019230769230769232
Hawaii: 0.7276142767689417
Idaho: 0.2926980198019802
Illinois: 0.3216988591437931
Indiana: 0.16289351322509338
Iowa: 0.37317993888189827
Kansas: 0.19275717789640745
Kentucky: 0.24891540130151843
Louisiana: 0.306158065320369
Maine: 0.09098939929328621
Marshall Islands: 1.0
Maryland: 0.34485563237088246
Massachusetts: 0.24600422832980973
Michigan: 0.49466818511233046
Minnesota: 0.3041659787573606
Mississippi: 0.0976123595505618
Missouri: 0.19318715740015663
Montana: 0.07088122605363985
Nebraska: 0.1738180373460469
Nevada: 0.08027006751687922
New Hampshire: 0.32723239046034386
New Jersey: 0.092705167

Awesome! Looks like there's a lot of variation!

Let's preemptively play around with plotting this, since we have some lat/long data free.

In [12]:
import folium

In [13]:
# note that coordinates can be provided to folium as an array or as a tuple
test_coords = [float(test_json2['places'][0]['latitude']), float(test_json2['places'][0]['longitude'])]
test_map = folium.Map(location=test_coords, zoom_start=4, tiles='Stamen Toner')
# There are 2 ways to add a marker to a map (that don't result in a deprecated error), 
# but I'm using the latter because it allows me to save the marker and display the map
# in a separate cell. Interestingly, things like `popup` show up as "children" of the 
# Marker instance, which is itself a "child" of the Map instance
# test_map.add_children(folium.Marker(location=test_coords, popup="Some doctor's office"))
test_marker = folium.Marker(location=test_coords, popup="Some doctor's office").add_to(test_map)
# note: the return value of the `add_to` method is the Marker itself, so this still works. 
# It's possible that it returns an error, though?

In [14]:
type(test_marker) # test_map  # how pretty!

folium.map.Marker

In [15]:
print(test_map.to_json())  # note that to_json loses critical data like the location of a Marker
test_map

{"name": "Map", "id": "cbdcad850898495b926c9d58f3f65f46", "children": {"stamentoner": {"name": "TileLayer", "id": "4eed6b489b5e4bd4bb55dfc9adf4687e", "children": {}}, "marker_376af66dcaf64668b898fddd0a33f647": {"name": "Marker", "id": "376af66dcaf64668b898fddd0a33f647", "children": {"popup_3bf54b225247492bb18b51db5508c810": {"name": "Popup", "id": "3bf54b225247492bb18b51db5508c810", "children": {}}}}}}


In [15]:
states = full_df['Business.State.Territory'].value_counts()
print(len(states))
states

57


California                        29300
Florida                           28414
Texas                             28075
Pennsylvania                      22511
Wisconsin                         19578
Michigan                          19318
Ohio                              18900
Minnesota                         18171
North Carolina                    17921
Illinois                          17706
New York                          16603
Indiana                           13119
Missouri                          12770
Massachusetts                     11825
Virginia                          10955
Arizona                            9459
Washington                         9101
New Jersey                         8554
Georgia                            8280
Maryland                           7377
Kansas                             6931
Tennessee                          6504
Oregon                             6160
South Carolina                     5773
Iowa                               5563


In [77]:
full_df['Program.Type'].value_counts()

Medicare             404718
Medicare/Medicaid     17811
Name: Program.Type, dtype: int64

In [17]:
full_df['Product.Setting'].value_counts()

Ambulatory    373878
Inpatient      48651
Name: Product.Setting, dtype: int64

Interesting...I could see if the top vendors differ significantly between the product settings

In [26]:
def print_top_vendors(df):
    """Given a df, print the top 5 vendors with the most attestations"""
    print(df.iloc[0]['Product.Setting'])
    sorted_vendors = df['Vendor.Name'].value_counts().sort_values(ascending=False)
    print(sorted_vendors.head())
    return False

full_df.groupby('Product.Setting').apply(print_top_vendors)

Ambulatory
Epic Systems Corporation    100277
Cerner Corporation           91264
Allscripts                   27348
eClinicalWorks LLC           18922
GE Healthcare                13635
Name: Vendor.Name, dtype: int64
Ambulatory
Epic Systems Corporation    100277
Cerner Corporation           91264
Allscripts                   27348
eClinicalWorks LLC           18922
GE Healthcare                13635
Name: Vendor.Name, dtype: int64
Inpatient
Epic Systems Corporation            20781
Cerner Corporation                  13664
MEDITECH                             3042
Sunquest Information Systems Inc     2788
McKesson                             1078
Name: Vendor.Name, dtype: int64


Product.Setting
Ambulatory    False
Inpatient     False
dtype: bool

The results are interesting. Looks like Epic and Cerner handily top both lists, but the next few are different.

The first group being printed twice is a result of the following warnings (from the pandas docs):
Warning In the current implementation apply calls func twice on the first group to decide whether it can take a fast or slow code path. This can lead to unexpected behavior if func has side-effects, as they will take effect twice for the first group.

In [22]:
full_df['Product.Certification.Edition.Yr'].value_counts()

2014    422529
Name: Product.Certification.Edition.Yr, dtype: int64

In [23]:
full_df.shape

(422529, 22)

In [159]:
len(full_df['NPI'].value_counts())

229452

Check that there are no hospitals in zip codes which span multiple states. Using zip code level granularity for analysis, but ignoring the fact that there are 13 (?) zip codes which span multiple states could be an annoying error.

In [155]:
npi_groups = full_df.groupby('NPI')

In [157]:
len(npi_groups)

229452

Is the number of NPIs equal to the number of attestation IDs?

In [160]:
attestation_groups = full_df.groupby('Attestation_ID')

In [161]:
len(attestation_groups)

229453

WTF is this off by one? Why is there one more attestation ID than NPI? Which NPI has 2 attestation IDs? We think there are no null NPIs, so the NPI groupby should not be failing there. Alternatively is there an attestation ID which has neither an NPI nor a CCN?|

In [212]:
attestation_groups['NPI'].nunique().sort_values()

Attestation_ID
1         1
636591    1
636602    1
636603    1
636606    1
636612    1
636615    1
636616    1
636627    1
636629    1
636632    1
636636    1
636586    1
636642    1
636647    1
636653    1
636655    1
636659    1
636663    1
636664    1
636672    1
636673    1
636680    1
636687    1
636690    1
636644    1
636691    1
636584    1
636565    1
636467    1
         ..
318556    1
318551    1
318547    1
318427    1
318439    1
318446    1
318447    1
318451    1
318456    1
318460    1
318465    1
318469    1
318486    1
318487    1
318494    1
318497    1
318498    1
318505    1
318506    1
318507    1
318514    1
318520    1
318522    1
318527    1
318533    1
318537    1
318538    1
318539    1
321288    1
954115    1
Name: NPI, dtype: int64

In [4]:
test_multi = full_df.groupby(['NPI'])
# we want the number of unique attestations per NPI

In [17]:
print(test_multi['Attestation_ID'].nunique().sort_values().tail(1).index[0])
test_multi['Attestation_ID'].nunique().sort_values()

1962455832


NPI
1003000142    1
1669538674    1
1669538963    1
1669539649    1
1669539862    1
1669540100    1
1669540191    1
1669540308    1
1669540324    1
1669540464    1
1669540472    1
1669540522    1
1669538591    1
1669540530    1
1669540654    1
1669540761    1
1669541066    1
1669541157    1
1669541207    1
1669541504    1
1669541678    1
1669541702    1
1669541744    1
1669542833    1
1669542999    1
1669540613    1
1669543120    1
1669538047    1
1669536348    1
1669530648    1
             ..
1336146638    1
1336146612    1
1336146570    1
1336145739    1
1336145762    1
1336145770    1
1336145804    1
1336145838    1
1336145879    1
1336145895    1
1336145911    1
1336146000    1
1336146042    1
1336146059    1
1336146125    1
1336146208    1
1336146224    1
1336146257    1
1336146265    1
1336146273    1
1336146299    1
1336146315    1
1336146349    1
1336146448    1
1336146497    1
1336146513    1
1336146521    1
1336147834    1
1992999825    1
1962455832    2
Name: Attestation_ID

In [219]:
full_df[full_df['NPI']==1962455832]
# full_df[full_df['NPI']==1962455832]['Business.State.Territory']

,EHR.Certification.Number,Vendor.Name,EHR.Product.CHP.Id,EHR.Product.Name,EHR.Product.Version,Product.Classification,Product.Setting,Product.Certification.Edition.Yr,Attestation.Month,Attestation.Year,...,Specialty,Program.Year,Payment.Year,Provider.Stage.Number,Program.Type,Stage.2.Scheduled.2014,MU.Definition.2014,Attestation_ID,NPI,CCN
406552,1314E01RMR9BEAD,Cerner Corporation,CHP-019313,P2 Sentinel (Powered by Sensage),5,Modular EHR,Inpatient,2014,3,2016,...,NaN,2015,1.0,Stage 1,Medicare/Medicaid,1.0,NaN,919100,1962455832,450052.0
406553,1314E01RMR9BEAD,Cerner Corporation,CHP-025188,Powerchart (Clinical),2015.01.01,Modular EHR,Inpatient,2014,3,2016,...,NaN,2015,1.0,Stage 1,Medicare/Medicaid,1.0,NaN,919100,1962455832,450052.0
406554,1314E01RMR9BEAD,Cerner Corporation,CHP-025192,Powerchart (eRX),2015.01.01,Modular EHR,Inpatient,2014,3,2016,...,NaN,2015,1.0,Stage 1,Medicare/Medicaid,1.0,NaN,919100,1962455832,450052.0
406555,1314E01RMR9BEAD,Cerner Corporation,CHP-025193,Powerchart (All CQM's),2015.01.01,Modular EHR,Inpatient,2014,3,2016,...,NaN,2015,1.0,Stage 1,Medicare/Medicaid,1.0,NaN,919100,1962455832,450052.0
406556,1314E01RMR9BEAD,Cerner Corporation,CHP-025470,PathNet,2015.01.03,Modular EHR,Inpatient,2014,3,2016,...,NaN,2015,1.0,Stage 1,Medicare/Medicaid,1.0,NaN,919100,1962455832,450052.0
406557,1314E01RMR9BEAD,Cerner Corporation,CHP-028305,Powerchart (All CQM's),2015.01.08,Modular EHR,Inpatient,2014,3,2016,...,NaN,2015,1.0,Stage 1,Medicare/Medicaid,1.0,NaN,919100,1962455832,450052.0
406558,1314E01RMR9BEAD,Cerner Corporation,CHP-028310,HealthSentry,2015.08,Modular EHR,Inpatient,2014,3,2016,...,NaN,2015,1.0,Stage 1,Medicare/Medicaid,1.0,NaN,919100,1962455832,450052.0
406559,1314E01RMR9BEAD,Cerner Corporation,CHP-028318,Powerchart (Clinical),2015.01.08,Modular EHR,Inpatient,2014,3,2016,...,NaN,2015,1.0,Stage 1,Medicare/Medicaid,1.0,NaN,919100,1962455832,450052.0
406560,1314E01RMR9BEAD,Cerner Corporation,CHP-028328,Powerchart (eRX),2015.01.08,Modular EHR,Inpatient,2014,3,2016,...,NaN,2015,1.0,Stage 1,Medicare/Medicaid,1.0,NaN,919100,1962455832,450052.0
406561,1314E01RMR9BEAD,Cerner Corporation,CHP-028429,HealthSentry,2015.09,Modular EHR,Inpatient,2014,3,2016,...,NaN,2015,1.0,Stage 1,Medicare/Medicaid,1.0,NaN,919100,1962455832,450052.0


In [222]:
full_df.groupby('Vendor.Name')['EHR.Product.Name'].nunique().sort_values()

Vendor.Name
ACL Laboratories                              1
OEMR                                          1
OD Link                                       1
OCERIS Inc                                    1
OA Systems Inc                                1
Nuesoft Technologies Inc                      1
Nuance Communications Inc                     1
Nth Technologies Inc                          1
Novobi                                        1
Nortec Software Inc                           1
NoMoreClipboard LLC                           1
Nexus Clinical LLC                            1
NextServices Inc                              1
Nexsyis Inc                                   1
NexSched                                      1
NewYork-Presbyterian Hospital                 1
Networking Technology dba RxNT                1
NeoDeck Software                              1
NavisHealth Solutions Inc                     1
Navigating Cancer Inc                         1
NTT DATA Inc                

In [223]:
test = full_df.groupby(['Business.State.Territory', 'Vendor.Name'])